In [2]:
import pyprind
import pandas as pd
import os

In [9]:
pbar = pyprind.ProgBar(50000)
labels = {'pos': 1, 'neg': 0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './data/aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index = True)
            pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:03:13


In [10]:
df.columns = ['review', 'sentiment']

In [11]:
df.head(2)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1


In [12]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./data/movie_data.csv', index=False)

## bag_of_words model - an example

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
count = CountVectorizer()
docs = np.array(['The sun is shining',
                'The weather is sweet', 
                'The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)

In [17]:
count.vocabulary_

{u'and': 0,
 u'is': 1,
 u'shining': 2,
 u'sun': 3,
 u'sweet': 4,
 u'the': 5,
 u'weather': 6}

In [24]:
print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


### TF-IDF, term frequency-inverse document frequency

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision = 2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.56  0.56  0.    0.43  0.  ]
 [ 0.    0.43  0.    0.    0.56  0.43  0.56]
 [ 0.4   0.48  0.31  0.31  0.31  0.48  0.31]]


## cleaning text data

In [30]:
df.loc[1,'review']

'Actor turned director Bill Paxton follows up his promising debut, the Gothic-horror "Frailty", with this family friendly sports drama about the 1913 U.S. Open where a young American caddy rises from his humble background to play against his Bristish idol in what was dubbed as "The Greatest Game Ever Played." I\'m no fan of golf, and these scrappy underdog sports flicks are a dime a dozen (most recently done to grand effect with "Miracle" and "Cinderella Man"), but some how this film was enthralling all the same.<br /><br />The film starts with some creative opening credits (imagine a Disneyfied version of the animated opening credits of HBO\'s "Carnivale" and "Rome"), but lumbers along slowly for its first by-the-numbers hour. Once the action moves to the U.S. Open things pick up very well. Paxton does a nice job and shows a knack for effective directorial flourishes (I loved the rain-soaked montage of the action on day two of the open) that propel the plot further or add some unexpec

In [54]:
# strip
import re
def preprocessor(text):
    # remove html markup
    text = re.sub('<[^>]*>', '', text)
    # organize emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [55]:
preprocessor('</a>This :-) is :( a test :-)!')

'this is a test :) :( :)'

In [56]:
df['review'] = df['review'].apply(preprocessor)

In [57]:
# transform doc into tokens
def tokenizer(text):
    return text.split()

In [58]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [59]:
# tokenization with word stemming, using Porter stemmer
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [61]:
tokenizer_porter('runners like running and thus they run') # notice thus was tokenized to be thu

[u'runner', 'like', u'run', 'and', u'thu', 'they', 'run']

In [62]:
# stop-words from NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

[nltk_data] Downloading package stopwords to /Users/Jack/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['runner', u'like', u'run', u'run', 'lot']

## train a logistic regression model for document classification

In [64]:
# prepare data
x_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
x_test = df.loc[25000:, 'review'].values
y_test = df.loc[2500:, 'sentiment'].values

## out-of-core learning

In [65]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [66]:
next(stream_docs(path='./data/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [67]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [68]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [69]:
vect = HashingVectorizer(decode_error='ignore',
                        n_features = 2**21,
                        preprocessor = None,
                        tokenizer = tokenizer)
clf = SGDClassifier(loss='log', random_state = 1, n_iter = 1)
doc_stream = stream_docs(path = './data/movie_data.csv')

In [70]:
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    x_train, y_train = get_minibatch(doc_stream, size = 1000)
    if not x_train:
        break
    x_train = vect.transform(x_train)
    clf.partial_fit(x_train, y_train, classes = classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:12


In [71]:
x_test, y_test = get_minibatch(doc_stream, size=5000)
x_test = vect.transform(x_test)
print('accuracy: %.3f' % clf.score(x_test, y_test))

accuracy: 0.805
